In [1]:
import pandas as pd
from data_preparation import  clean
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout, Bidirectional
import numpy as np
from sklearn.metrics import  classification_report
from emoji import UNICODE_EMOJI

In [2]:
data = pd.read_csv("offenssivelabelfinal.csv")
data.head()

,comment,off
0,هادك المجرم لي قتل الطفل عدنان خصو لاعدام,1
1,الطفل عدنان الله يرحمو ولكن لن يرتاح عدنان...,1
2,خاصو الاعدام بدون نقاش,1
3,نطالب بالإعدام ثم الإعدام,1
4,راه مزال كنتسناو في الاعدام ابغينا الاعدام وف...,1


In [4]:
data_train = pd.read_csv("train.csv")
data_train

,Unnamed: 0,comment,off
0,2908,فنانين الكبت والفساد .عقلية جنسية لا غير. العف...,1
1,1055,الدعارة هربت منها في المحمدية و سكنت في بوزنيق...,1
2,181,كون غير خريتي و مدرتيش هادشي,1
3,4313,لا حول ولا قوة الا بالله العلي العظيم لا حول و...,0
4,228,الله يرحم الوالدين عندي مشكل كبيييير و محتاجة ...,0
...,...,...,...
6414,79,هدى جمعاك شعر كحل,0
6415,3927,تفو.. على أغنية زملاوية بحال مولاها,1
6416,5955,كون خرجات ها.ي كالت انا كنشطح كون بان ليك مساع...,1
6417,6936,الفديو زوين.,0


In [5]:
data_test = pd.read_csv("test.csv")
data_test

,Unnamed: 0,comment,off
0,6139,تبهديل تسؤلو البنات في الشارع,1
1,3447,انا مستعد بي مسعدة بسيطة في وجه اللة ربي يادير...,0
2,6844,لاحول ولاقوة الا بالله العلي العظيم,0
3,2512,حتى هاد العائلة لي جالس وسط منهوم خص يدلمهوم ا...,1
4,2000,لا حول ولا قوة الا بالله اش هذا المستوى اصلا ر...,1
...,...,...,...
1600,6470,لي جا من عند بيمو اخبط لايك 😹😹😹😹,0
1601,6198,ونتساءل لمادا الله ارسل لنا فيروس كورونا,1
1602,3009,خانزة كيلوط وا الرويجل سير دير الرقية نصحك هاد...,1
1603,647,سير الله يعفو اعليك,0


In [6]:
data_train["clean_comment"]=data_train["comment"].apply(lambda row: clean(row))
data_test["clean_comment"]=data_test["comment"].apply(lambda row: clean(row))

In [7]:
def replace_emoji(sent):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', sent)

In [8]:
def is_emoji(s):
    return s in UNICODE_EMOJI['en']
# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

In [9]:
#data["clean_comment_nemoji"]=data["clean_comment"].apply(lambda row: replace_emoji(row))
data_train["clean_comment_nemoji"]=data_train["clean_comment"].apply(lambda row: replace_emoji(row))
data_test["clean_comment_nemoji"]=data_test["clean_comment"].apply(lambda row: replace_emoji(row))

In [10]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data_train['clean_comment_nemoji'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 23924 unique tokens.


In [11]:
X_train = tokenizer.texts_to_sequences(data_train['clean_comment_nemoji'].values)
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train.shape)

Shape of data tensor: (6419, 250)


In [12]:
X_test = tokenizer.texts_to_sequences(data_test['clean_comment_nemoji'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (1605, 250)


In [14]:
data_test['clean_comment_nemoji'][8]

'سير الله العن لمرباكش تربيه حسنه'

In [15]:
Y_train = pd.get_dummies(data_train['off']).values
print('Shape of label tensor:', Y_train.shape)

Shape of label tensor: (6419, 2)


In [26]:
Y_test = pd.get_dummies(data_test['off']).values
print('Shape of label tensor:', Y_test.shape)

Shape of label tensor: (1605, 2)


In [17]:
#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6419, 250) (6419, 2)
(1605, 250) (6419, 2)


In [20]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 5,080,602
Trainable params: 5,080,602
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
epochs = 15
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

W0329 23:14:56.772566 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_train_function.<locals>.train_function at 0x0000029D498945E0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Epoch 1/15
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
91/91 [==============================] - ETA: 0s - loss: 0.6131 - accuracy: 0.6507

W0329 23:15:36.871366 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000029D4BCC3280> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
91/91 [==============================] - 41s 419ms/step - loss: 0.6131 - accuracy: 0.6507 - val_loss: 0.5053 - val_accuracy: 0.7555
Epoch 2/15
91/91 [==============================] - 38s 418ms/step - loss: 0.2826 - accuracy: 0.8828 - val_loss: 0.4876 - val_accuracy: 0.7897
Epoch 3/15
91/91 [==============================] - 42s 458ms/step - loss: 0.1065 - accuracy: 0.9633 - val_loss: 0.5856 - val_accuracy: 0.7819
Epoch 4/15
91/91 [==============================] - 40s 439ms/step - loss: 0.0485 - accuracy: 0.9851 - val_loss: 0.6432 - val_accuracy: 0.7835
Epoch 5/15
91/91 [==============================] - 41s 451ms/step - loss: 0.0306 - accuracy: 0.9920 - val_loss: 0.7190 - val_accuracy: 

In [22]:
prediction = model.predict(X_test)

W0329 23:19:23.646478 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_predict_function.<locals>.predict_function at 0x0000029D4BF14040> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [23]:
y_pred_bool = np.argmax(prediction, axis=1)
y_pred = (prediction > 0.5)
y_pred

array([[ True, False],
       [ True, False],
       [ True, False],
       ...,
       [False,  True],
       [False,  True],
       [ True, False]])

In [29]:
Y_test

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [30]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.81      0.76       717
           1       0.83      0.73      0.78       888

   micro avg       0.77      0.77      0.77      1605
   macro avg       0.77      0.77      0.77      1605
weighted avg       0.78      0.77      0.77      1605
 samples avg       0.77      0.77      0.77      1605



In [31]:
y_pred_bool

array([0, 0, 0, ..., 1, 1, 0], dtype=int64)

In [32]:
y_test_bool=np.argmax(Y_test, axis=1)
y_test_bool

array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [33]:
print(classification_report(y_test_bool, y_pred_bool, digits=4))

              precision    recall  f1-score   support

           0     0.7113    0.8145    0.7594       717
           1     0.8304    0.7331    0.7787       888

    accuracy                         0.7695      1605
   macro avg     0.7708    0.7738    0.7691      1605
weighted avg     0.7772    0.7695    0.7701      1605



# BILSTM

In [74]:
model1 = Sequential()
model1.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model1.add(SpatialDropout1D(0.2))
model1.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model1.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model1.add(Dense(2, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d_6 (Spatia  (None, 250, 100)         0         
 lDropout1D)                                                     
                                                                 
 bidirectional_4 (Bidirectio  (None, 250, 200)         160800    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 2)                 402       
                                                      

In [75]:
epochs = 15
batch_size = 64

history1 = model1.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

W0331 23:50:55.089491 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_train_function.<locals>.train_function at 0x0000029DB5A52EE0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Epoch 1/15
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
91/91 [==============================] - ETA: 0s - loss: 0.6013 - accuracy: 0.6425  

W0401 10:07:15.387005 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000029D699031F0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
91/91 [==============================] - 36988s 410s/step - loss: 0.6013 - accuracy: 0.6425 - val_loss: 0.5014 - val_accuracy: 0.7632
Epoch 2/15
91/91 [==============================] - 2857s 31s/step - loss: 0.2448 - accuracy: 0.9022 - val_loss: 0.5230 - val_accuracy: 0.7695
Epoch 3/15
91/91 [==============================] - 2789s 31s/step - loss: 0.0812 - accuracy: 0.9716 - val_loss: 0.6854 - val_accuracy: 0.7726
Epoch 4/15
91/91 [==============================] - 2302s 25s/step - loss: 0.0365 - accuracy: 0.9886 - val_loss: 0.7309 - val_accuracy: 0.7632


In [76]:
prediction1 = model1.predict(X_test)

W0401 12:19:50.479997 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_predict_function.<locals>.predict_function at 0x0000029D69749040> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [77]:
y_pred_bool1 = np.argmax(prediction1, axis=1)
y_pred1 = (prediction1 > 0.5)
y_pred1

array([[ True, False],
       [ True, False],
       [ True, False],
       ...,
       [False,  True],
       [False,  True],
       [ True, False]])

In [78]:
Y_test

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [79]:
print(classification_report(Y_test, y_pred1, digits=4))

              precision    recall  f1-score   support

           0     0.7241    0.7978    0.7591       717
           1     0.8221    0.7545    0.7868       888

   micro avg     0.7738    0.7738    0.7738      1605
   macro avg     0.7731    0.7761    0.7730      1605
weighted avg     0.7783    0.7738    0.7745      1605
 samples avg     0.7738    0.7738    0.7738      1605



In [80]:
y_pred_bool1

array([0, 0, 0, ..., 1, 1, 0], dtype=int64)

In [81]:
y_test_bool=np.argmax(Y_test, axis=1)
y_test_bool

array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [82]:
print(classification_report(y_test_bool, y_pred_bool1, digits=4))

              precision    recall  f1-score   support

           0     0.7241    0.7978    0.7591       717
           1     0.8221    0.7545    0.7868       888

    accuracy                         0.7738      1605
   macro avg     0.7731    0.7761    0.7730      1605
weighted avg     0.7783    0.7738    0.7745      1605



# with emojis

In [46]:
def is_emoji(s):
    return s in UNICODE_EMOJI['en']
# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

In [47]:
#data["clean_comment_nemoji"]=data["clean_comment"].apply(lambda row: replace_emoji(row))
data_train["clean_comment_emoji"]=data_train["clean_comment"].apply(lambda row: add_space(row))
data_test["clean_comment_emoji"]=data_test["clean_comment"].apply(lambda row: add_space(row))

In [48]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data_train['clean_comment_emoji'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 24095 unique tokens.


In [49]:
X_train = tokenizer.texts_to_sequences(data_train['clean_comment_emoji'].values)
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train.shape)

Shape of data tensor: (6419, 250)


In [50]:
X_test = tokenizer.texts_to_sequences(data_test['clean_comment_emoji'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (1605, 250)


In [51]:
data_test['clean_comment_emoji'][8]

'سير الله العن لمرباكش تربيه حسنه'

In [52]:
Y_train = pd.get_dummies(data_train['off']).values
print('Shape of label tensor:', Y_train.shape)

Shape of label tensor: (6419, 2)


In [53]:
Y_test = pd.get_dummies(data_test['off']).values
print('Shape of label tensor:', Y_test.shape)

Shape of label tensor: (1605, 2)


In [54]:
#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6419, 250) (6419, 2)
(1605, 250) (1605, 2)


In [55]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d_4 (Spatia  (None, 250, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
Total params: 5,080,602
Trainable params: 5,080,602
Non-trainable params: 0
_________________________________________________________________
None


In [56]:
epochs = 15
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

W0330 12:05:28.146663 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_train_function.<locals>.train_function at 0x0000029D412BC820> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Epoch 1/15
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
91/91 [==============================] - ETA: 0s - loss: 0.6149 - accuracy: 0.6458

W0330 12:07:07.256742 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000029D660A0670> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
91/91 [==============================] - 100s 1s/step - loss: 0.6149 - accuracy: 0.6458 - val_loss: 0.5010 - val_accuracy: 0.7492
Epoch 2/15
91/91 [==============================] - 95s 1s/step - loss: 0.2898 - accuracy: 0.8811 - val_loss: 0.5037 - val_accuracy: 0.7882
Epoch 3/15
91/91 [==============================] - 96s 1s/step - loss: 0.1050 - accuracy: 0.9619 - val_loss: 0.6283 - val_accuracy: 0.7710
Epoch 4/15
91/91 [==============================] - 101s 1s/step - loss: 0.0492 - accuracy: 0.9853 - val_loss: 0.7116 - val_accuracy: 0.7741


In [57]:
prediction = model.predict(X_test)

W0330 12:12:01.267088 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_predict_function.<locals>.predict_function at 0x0000029D6615C310> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [58]:
y_pred_bool = np.argmax(prediction, axis=1)
y_pred = (prediction > 0.5)
y_pred

array([[ True, False],
       [ True, False],
       [ True, False],
       ...,
       [False,  True],
       [False,  True],
       [ True, False]])

In [59]:
Y_test

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [60]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.86      0.77       717
           1       0.86      0.69      0.77       888

   micro avg       0.77      0.77      0.77      1605
   macro avg       0.78      0.78      0.77      1605
weighted avg       0.79      0.77      0.77      1605
 samples avg       0.77      0.77      0.77      1605



In [61]:
y_pred_bool

array([0, 0, 0, ..., 1, 1, 0], dtype=int64)

In [62]:
y_test_bool=np.argmax(Y_test, axis=1)
y_test_bool

array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [63]:
print(classification_report(y_test_bool, y_pred_bool, digits=4))

              precision    recall  f1-score   support

           0     0.6912    0.8647    0.7683       717
           1     0.8630    0.6881    0.7657       888

    accuracy                         0.7670      1605
   macro avg     0.7771    0.7764    0.7670      1605
weighted avg     0.7862    0.7670    0.7668      1605



# BILSTM

In [64]:
model1 = Sequential()
model1.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model1.add(SpatialDropout1D(0.2))
model1.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model1.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model1.add(Dense(2, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d_5 (Spatia  (None, 250, 100)         0         
 lDropout1D)                                                     
                                                                 
 bidirectional_2 (Bidirectio  (None, 250, 200)         160800    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 2)                 402       
                                                      

In [65]:
epochs = 15
batch_size = 64

history1 = model1.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

W0330 12:12:06.199584 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_train_function.<locals>.train_function at 0x0000029D6E0445E0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Epoch 1/15
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
91/91 [==============================] - ETA: 0s - loss: 0.6020 - accuracy: 0.6367 

W0330 12:30:14.169066 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000029DB15E3550> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
91/91 [==============================] - 1093s 12s/step - loss: 0.6020 - accuracy: 0.6367 - val_loss: 0.4875 - val_accuracy: 0.7648
Epoch 2/15
91/91 [==============================] - 1509s 17s/step - loss: 0.2473 - accuracy: 0.9048 - val_loss: 0.5271 - val_accuracy: 0.7819
Epoch 3/15
91/91 [==============================] - 2735s 30s/step - loss: 0.0934 - accuracy: 0.9683 - val_loss: 0.6332 - val_accuracy: 0.7664
Epoch 4/15
91/91 [==============================] - 2802s 31s/step - loss: 0.0423 - accuracy: 0.9884 - val_loss: 0.8292 - val_accuracy: 0.7679


In [66]:
prediction1 = model1.predict(X_test)

W0330 14:27:45.654215 28628 ag_logging.py:146] AutoGraph could not transform <function Model.make_predict_function.<locals>.predict_function at 0x0000029D67563820> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [67]:
y_pred_bool1 = np.argmax(prediction1, axis=1)
y_pred1 = (prediction1 > 0.5)
y_pred1

array([[ True, False],
       [ True, False],
       [ True, False],
       ...,
       [False,  True],
       [ True, False],
       [ True, False]])

In [68]:
Y_test

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [69]:
print(classification_report(Y_test, y_pred1, digits=4))

              precision    recall  f1-score   support

           0     0.7219    0.7964    0.7573       717
           1     0.8206    0.7523    0.7850       888

   micro avg     0.7720    0.7720    0.7720      1605
   macro avg     0.7713    0.7743    0.7711      1605
weighted avg     0.7765    0.7720    0.7726      1605
 samples avg     0.7720    0.7720    0.7720      1605



In [70]:
y_pred_bool1

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [71]:
y_test_bool=np.argmax(Y_test, axis=1)
y_test_bool

array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [73]:
print(classification_report(y_test_bool, y_pred_bool1, digits=4))

              precision    recall  f1-score   support

           0     0.7219    0.7964    0.7573       717
           1     0.8206    0.7523    0.7850       888

    accuracy                         0.7720      1605
   macro avg     0.7713    0.7743    0.7711      1605
weighted avg     0.7765    0.7720    0.7726      1605

